# ANOVA de un factor y Prueba Kruskal-Wallis

El análisis de varianza (ANOVA) describe a un conjunto técnicas estadísticas que se utilizan para comparar la media entre dos o más grupos. La hipótesis nula es que todas las medias son iguales mientras que la hipótesis alternativa plantea que al menos una es diferente.

El análisis de varianza de un factor implica una variable dependiente continua y una variable independiente (llamada factor) que tiene cierto número de niveles. Los diferentes niveles de la variable independiente corresponden a diferente grupos o condiciones. Cuando existen dos o más factores el análisis se le llama análisis de varianza de n factores. Si el conjunto de variables independientes incluye variables cuantitativas (covariables), a la técnica se le llama análisis de covarianza (ANCOVA). Si hay dos o más variables dependientes se le conoce como MANOVA.

El análisis de varianza recibe su nombre debido a que compara la variabilidad entre grupos con la variabilidad dentro de los grupos. Se calcula un valor F que representa la variabilidad entre los grupos dividida entre la variabilidad dentro de los grupos. Un valor grande de la F indica que hay más variabilidad entre los grupos (causados por la variable independiente) que la que hay dentro de cada grupo (atribuida al azar).

Un valor significativo en la F implica que se rechaza la hipótesis nula, es decir, que al menos una media es diferente. Para determinar qué grupo es diferente se deben llevar a cabo pruebas post-hoc o realizar comparaciones específicas. 

El ANOVA de un factor requiere el cumplimiento de varios supuestos: las muestras son independientes, la variable dependiente está normalmente distribuida en cada grupo, y las varianzas en cada grupo son iguales. Debido a que la ANOVA se considera como una prueba robusta, variaciones en el cumplimiento de los supuestos de normalidad y homogeneidad de varianzas no afectan de manera importante los resultados si el tamaño de las muestras es igual. 

**Caso de uso**
Una cadena de comida rápida planea agregar un nuevo producto a su menú, pero están indecisos entre tres posibles campañas de mercadotecnica. Como un experimento, el producto fue introducido en varias ubicaciones seleccionadas aleatoriamente utilizando diferentes campañas. Se registraron las ventas del nuevo producto por las primeras cuatro semanas. El archivo "marketing.csv" contiene las siguientes variables:  
- MarketID: identificador del mercado
- MarketSize: tamaño del mercado de acuerdo a las ventas
- LocationID: identificador de la ubicación de la tienda
- AgeOfStore: antigüedad de la tienda en años
- Promotion: promoción (1 de 3) que fue probada
- week: semana en que se llevó a cabo la promoción.
- SalesInThousands: ventas para una ubicación específica (LocationID), promoción (Promotion) y semana (week).

Para comparar las ventas de acuerdo a las diferentes promociones, utilizaremos una ANOVA  
Hipótesis nula: todas las medias son iguales.  
Hipótesis alternativa: al menos un par es diferente.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../datasets/marketing.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548 entries, 0 to 547
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MarketID          548 non-null    int64  
 1   MarketSize        548 non-null    object 
 2   LocationID        548 non-null    int64  
 3   AgeOfStore        548 non-null    int64  
 4   Promotion         548 non-null    int64  
 5   week              548 non-null    int64  
 6   SalesInThousands  548 non-null    float64
dtypes: float64(1), int64(5), object(1)
memory usage: 30.1+ KB


In [4]:
df['Promotion'].value_counts()

Promotion
3    188
2    188
1    172
Name: count, dtype: int64

In [5]:
df.sample(4)

,MarketID,MarketSize,LocationID,AgeOfStore,Promotion,week,SalesInThousands
285,6,Medium,507,5,2,2,17.34
301,6,Medium,511,24,3,2,39.41
73,2,Small,106,5,3,2,62.19
271,6,Medium,503,2,2,4,37.20


## Supuesto de igualdad de varianzas

In [4]:
from scipy.stats import levene

def compare_variances(data, group_column, measure_column, groups=None, alpha=0.05):
    """
    Realiza e interpreta una prueba de Levene para comparar varianzas entre múltiples grupos.   
    Args:
        data: DataFrame con los datos
        group_column: Nombre de la columna que identifica los grupos
        measure_column: Nombre de la columna con la variable a comparar
        groups: Lista de grupos a comparar (si es None, usa todos los grupos encontrados)
        alpha: Nivel de significancia (default: 0.05)
    Returns:
        p-value
    """
    # Identificar grupos si no se especifican
    grupos_disponibles = data[group_column].unique()
    if len(grupos_disponibles) < 2:
        raise ValueError(f"La columna {group_column} debe tener al menos dos grupos")
    
    # Usar grupos especificados o todos los disponibles
    grupos = groups if groups is not None else grupos_disponibles
    
    # Verificar que todos los grupos especificados existen
    grupos_invalidos = set(grupos) - set(grupos_disponibles)
    if grupos_invalidos:
        raise ValueError(f"Grupos no encontrados en los datos: {grupos_invalidos}")
    
    # Obtener datos de cada grupo
    datos_grupos = [data[data[group_column] == grupo][measure_column] for grupo in grupos]
    
    # Calcular estadísticas por grupo
    stats = pd.DataFrame({
        'n': [len(datos) for datos in datos_grupos],
        'varianza': [datos.var() for datos in datos_grupos],
        'desv_std': [datos.std() for datos in datos_grupos]
    }, index=grupos)
    
    # Realizar prueba de Levene
    stat, p_value = levene(*datos_grupos)
    
    # Preparar interpretación
    es_significativo = p_value < alpha
    interpretacion = (f"Prueba de Levene para {len(grupos)} grupos\n"
                     f"Estadístico de Levene: {stat:.4f}\n"
                     f"Valor p: {p_value:.4f}\n\n"
                     "Estadísticas por grupo:\n")
    
    # Añadir estadísticas de cada grupo
    for grupo in grupos:
        n = stats.loc[grupo, 'n']
        var = stats.loc[grupo, 'varianza']
        std = stats.loc[grupo, 'desv_std']
        interpretacion += f"{grupo}: n={n}, varianza={var:.2f}, desv. estándar={std:.2f}\n"
    
    # Añadir conclusión
    interpretacion += f"\nConclusión: {'Se rechaza' if es_significativo else 'No se rechaza'} "
    interpretacion += "la hipótesis nula de igualdad de varianzas"
    interpretacion += (" (hay diferencias significativas en las varianzas entre los grupos)" 
                      if es_significativo 
                      else "\n (no hay evidencia de diferencias significativas en las varianzas)")
    
    print(interpretacion)
    return p_value

In [8]:
resultados_levene = compare_variances(df,'Promotion', 'SalesInThousands')

Prueba de Levene para 3 grupos
Estadístico de Levene: 1.2697
Valor p: 0.2818

Estadísticas por grupo:
3: n=188, varianza=281.11, desv. estándar=16.77
2: n=188, varianza=228.28, desv. estándar=15.11
1: n=172, varianza=274.03, desv. estándar=16.55

Conclusión: No se rechaza la hipótesis nula de igualdad de varianzas
 (no hay evidencia de diferencias significativas en las varianzas)


## Supuesto de normalidad

In [26]:
from scipy import stats
import pandas as pd

def test_ks_normality(df, group_column, measure_column):
    """
    Realiza la prueba de Kolmogorov-Smirnov para evaluar normalidad por grupos
    Args:
        df: DataFrame con los datos
        group_col: Nombre de la columna de agrupación
        value_col: Nombre de la columna con valores a evaluar
    Returns:
        DataFrame con resultados de la prueba por grupo
    """
    results = []
    for group in df[group_column].unique():
        data = df[df[group_column] == group][measure_column]
        stat, pval = stats.kstest(data, 'norm', args=(data.mean(), data.std()))
        results.append({'grupo': group,'estadistico_ks': stat.round(4),'valor_p': pval.round(4)})
        
    return pd.DataFrame(results)

In [27]:
normalidad = test_ks_normality(df,'Promotion', 'SalesInThousands')
normalidad

,grupo,estadistico_ks,valor_p
0,3,0.1459,0.0006
1,2,0.1425,0.0009
2,1,0.1227,0.0103


Sin embargo, varios estudios muestran que la prueba ANOVA es robusta ante violaciones de la normalidad si:
- Los grupos tienen un tamaño similar
- Hay por lo menos 40 observaciones en cada grupo.

## ANOVA

In [28]:
from scipy.stats import f_oneway

def realizar_anova(df, group_column, measure_column, alpha=0.05):
    """
    Realiza prueba ANOVA de una vía para comparar medias entre grupos.
    Parámetros:
    df (pandas.DataFrame): DataFrame con los datos
    grupo_col (str): Nombre de la columna que contiene los grupos
    valor_col (str): Nombre de la columna con los valores a comparar
    alpha (float): Nivel de significancia (default 0.05)
    Retorna:
    p-value
    """
    # Crear valores por grupo
    grupos = df[group_column].unique()
    valores_grupos = [df[df[group_column] == grupo][measure_column] for grupo in grupos]
    
    # Realizar prueba ANOVA
    statistic, p_value = f_oneway(*valores_grupos)
    
    # Determinar resultado
    resultado = "Se rechaza la hipótesis nula: hay al menos una diferencia significativa entre los grupos." if p_value < alpha else "No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar que hay diferencias significativas entre los grupos."

    print(f'Estadistico {statistic:.4f}')
    print(f'Valor p: {p_value:.4f}')
    print(resultado)
    
    return  p_value

In [29]:
resultado = realizar_anova(df,'Promotion', 'SalesInThousands')

Estadistico 21.9535
Valor p: 0.0000
Se rechaza la hipótesis nula: hay al menos una diferencia significativa entre los grupos.


La tabla ANOVA no indica qué grupo es diferente al resto, sin embargo, las pruebas post-hoc son útiles para detectar qué grupo es diferente al resto. 
- LSD/DMS (Diferencia menos significativa): Es el equivalente a múltiples pruebas t, no se hacen correcciones y los resultados no son precisos.
- Bonferroni: Corrige el nivel de significancia dividiéndolo entre el número de grupos. Es preferible cuando son pocas comparaciones. 
- Tukey: Preferible cuando son muchas comparaciones. Es deseable que el tamaño de cada grupo sea igual. 
- REGWQ (Ryan-Einot-Gabriel-Welsh): Recomendable, pero se debe evitar cuando las muestras son de diferente tamaño.
- Dunnett: Es apropiada cuando se desea comparar con un grupo de control.
- Gabriel: Apropiada cuando el tamaño de las muestras es ligeramente diferente.
- GT2 de Hochberg: Apropiadas cuando el tamaño de las muestras es muy diferente
- Games- Howell: Recomendable cuando las varianzas son diferentes.

La librería statsmodel permite realizar la prueba de Tukey, Bonferroni y Dunnet

In [48]:
def post_hoc_test(data, group_column, value_column, test_type='tukey', control_group=None, alpha=0.05):
    """
    Realiza pruebas post hoc múltiples: Tukey o Bonferroni.
    Parámetros:
    data: DataFrame que contiene los datos
    group_column: Nombre de la columna que contiene los grupos
    value_column: Nombre de la columna que contiene los valores a comparar
    test_type: Tipo de prueba ('tukey', 'bonferroni')
    alpha : Nivel de significancia (por defecto 0.05)
    Retorna:
    Objeto con resultados de la prueba seleccionada
    """
    from statsmodels.stats.multicomp import MultiComparison
    from statsmodels.stats.multicomp import pairwise_tukeyhsd
    
    mc = MultiComparison(data[value_column], data[group_column])
    
    if test_type.lower() == 'tukey':
        print(mc.tukeyhsd(alpha=alpha))
    
    elif test_type.lower() == 'bonferroni':
        print(mc.allpairtest(stats.ttest_ind, method='bonf')[0])
    
    else:
        raise ValueError("Tipo de prueba no válido. Usar 'tukey' o 'bonferroni'")

In [49]:
resultado_tukey = post_hoc_test(df, "Promotion", "SalesInThousands", test_type='tukey')
resultado_tukey

 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower    upper  reject
-----------------------------------------------------
     1      2 -10.7696    0.0 -14.7738 -6.7654   True
     1      3  -2.7345 0.2444  -6.7388  1.2697  False
     2      3   8.0351    0.0   4.1208 11.9493   True
-----------------------------------------------------


In [55]:
resultado_bonferroni = post_hoc_test(df, "Promotion", "SalesInThousands", test_type='bonferroni')
resultado_bonferroni

Test Multiple Comparison ttest_ind 
FWER=0.05 method=bonf
alphacSidak=0.02, alphacBonf=0.017
group1 group2   stat   pval  pval_corr reject
---------------------------------------------
     1      2  6.4537    0.0       0.0   True
     1      3  1.5551 0.1208    0.3624  False
     2      3 -4.8814    0.0       0.0   True
---------------------------------------------


Ejemplo de redacción de conclusiones: 

>Se realizó un ANOVA de un factor para analizar cómo ________ influye en _________. Los resultados muestran que _________tiene un efecto significativo en __________, F(_,_) = ___, p = ___. Las comparaciones post-hoc utilizando el método de Tukey HSD indican que la media de ______ es significativamente diferente a la media de _________ y _______

## Prueba Kruskal-Wallis

In [63]:
from scipy.stats import kruskal
import pandas as pd

def kruskal_test(df: pd.DataFrame, group_col: str, value_col: str, alpha: float = 0.05) -> dict:
    """
    Realiza la prueba de Kruskal-Wallis para comparar múltiples grupos.
    Args:
        df: DataFrame con los datos
        group_col: Nombre de la columna que contiene los grupos
        value_col: Nombre de la columna con los valores a comparar
        alpha: Nivel de significancia (por defecto 0.05)  
    Returns:
        dict: Diccionario con los resultados de la prueba
    """
    # Obtener grupos únicos
    grupos = [df[df[group_col] == g][value_col] for g in df[group_col].unique()]
    
    # Realizar prueba Kruskal-Wallis
    statistic, p_value = kruskal(*grupos)
    
    # Preparar resultados
    resultados = {
        'estadistico': statistic,
        'p_valor': p_value,
        'alfa': alpha,
        'decision': 'Se rechaza H0' if p_value < alpha else 'No se rechaza H0',
        'interpretacion': ('Hay evidencia de diferencias significativas entre los grupos.' 
                         if p_value < alpha 
                         else 'No hay suficiente evidencia para afirmar diferencias significativas entre los grupos.')}
    
    return resultados

In [62]:
# Ejemplo de uso:
resultados = kruskal_test(df, 'Promotion', 'SalesInThousands')
print(f"Estadístico: {resultados['estadistico']:.4f}")
print(f"Valor p: {resultados['p_valor']:.4f}")
print(f"Decisión: {resultados['decision']}")
print(f"Interpretación: {resultados['interpretacion']}")

Estadístico: 53.2948
Valor p: 0.0000
Decisión: Se rechaza H0
Interpretación: Hay evidencia de diferencias significativas entre los grupos.
